In [1]:
import os, gc, cv2, torch, shutil
import numpy as np
from PIL import Image
from transformers import AutoImageProcessor, ZoeDepthForDepthEstimation

# ===== 1) 模型加载 =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "Intel/zoedepth-nyu-kitti"
processor = AutoImageProcessor.from_pretrained(model_name)
model = ZoeDepthForDepthEstimation.from_pretrained(model_name).to(device).eval()

# ===== 2) 路径 =====
base_dir = "/kaggle/input/kitti-dataset/depth_selection/val_selection_cropped"
img_dir = os.path.join(base_dir, "image")
gt_dir  = os.path.join(base_dir, "groundtruth_depth")

OUT_ROOT = "/kaggle/working/download/kitti"
for d in ["pred_raw_npy","pred_aligned_npy","pred_raw_png16","gt_npy","gt_png16"]:
    os.makedirs(os.path.join(OUT_ROOT, d), exist_ok=True)

# ===== 3) 工具 =====
def npy_save(path, arr): np.save(path, arr.astype(np.float32))
def mm_png_save(path, depth_m):
    depth_mm = np.clip(depth_m * 1000.0, 0, 65535).astype(np.uint16)
    cv2.imwrite(path, depth_mm)

def scale_match(pred, gt, mask):
    v = mask & np.isfinite(gt) & np.isfinite(pred) & (gt > 0)
    if not np.any(v): return 1.0
    p, g = pred[v], gt[v]
    den = float((p**2).sum())
    return float((g * p).sum() / den) if den > 0 else 1.0

# ===== 4) 主循环：保存 预测 + GT =====
img_files = sorted(f for f in os.listdir(img_dir) if f.endswith(".png"))
saved = 0

for img_file in img_files:
    # GT 文件名：将首个 "_image_" 换为 "_groundtruth_depth_"
    gt_file = img_file.replace("_image_", "_groundtruth_depth_", 1)
    rgb_path = os.path.join(img_dir, img_file)
    gt_path  = os.path.join(gt_dir,  gt_file)
    if not os.path.exists(gt_path):
        continue

    # 读取 RGB / GT（KITTI 16-bit PNG → 米）
    rgb_bgr = cv2.imread(rgb_path)
    if rgb_bgr is None:
        continue
    rgb = cv2.cvtColor(rgb_bgr, cv2.COLOR_BGR2RGB)

    gt_png = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED)
    if gt_png is None:
        continue
    gt_depth = gt_png.astype(np.float32) / 256.0
    mask = gt_depth > 0

    # 推理（ZoeDepth 输出米制）
    with torch.no_grad():
        inputs = processor(images=Image.fromarray(rgb), return_tensors="pt").to(device)
        outputs = model(**inputs)
        processed = processor.post_process_depth_estimation(
            outputs, source_sizes=[(rgb.shape[0], rgb.shape[1])]
        )
        pred_metric = processed[0]["predicted_depth"].squeeze().detach().cpu().numpy().astype(np.float32)

    # 尺寸对齐
    if pred_metric.shape != gt_depth.shape:
        pred_metric = cv2.resize(pred_metric, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

    # （可选）scale-only 对齐一份
    s = scale_match(pred_metric, gt_depth, mask)
    pred_aligned = pred_metric * s

    stem = os.path.splitext(img_file)[0]

    # 保存预测（原始 + 16-bit 毫米 PNG）
    npy_save(os.path.join(OUT_ROOT, "pred_raw_npy",     f"{stem}.npy"), pred_metric)
    mm_png_save(os.path.join(OUT_ROOT, "pred_raw_png16", f"{stem}.png"), pred_metric)

    # 保存预测（对齐后 .npy）
    npy_save(os.path.join(OUT_ROOT, "pred_aligned_npy", f"{stem}.npy"), pred_aligned)

    # 保存 GT（米 + 16-bit 毫米 PNG）
    npy_save(os.path.join(OUT_ROOT, "gt_npy",     f"{stem}.npy"), gt_depth)
    mm_png_save(os.path.join(OUT_ROOT, "gt_png16", f"{stem}.png"), gt_depth)

    saved += 1

    # 清理
    del rgb_bgr, rgb, gt_depth, pred_metric, pred_aligned, inputs, outputs, processed
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print(f"[KITTI ZoeDepth SAVE] saved pairs: {saved} / {len(img_files)}")

# ===== 5) 打包下载 =====
zip_base = "/kaggle/working/zoe_kitti_preds_and_gt"
shutil.make_archive(zip_base, "zip", OUT_ROOT)
print(f"打包完成: {zip_base}.zip")


2025-08-14 18:13:59.577656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755195239.782224      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755195239.853172      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

[KITTI ZoeDepth SAVE] saved pairs: 1000 / 1000
打包完成: /kaggle/working/zoe_kitti_preds_and_gt.zip
